# Step-by-Step Walkthrough for Asking User Support
A general recipe and step-by-step walkthrough for asking user support using LLMs' output token probabilities

## Define the Base and Children LLM Classes
For extracting token log probabilities more easily

In [2]:
import os
from pathlib import Path
from openai import OpenAI
from pprint import pprint
from abc import ABC, abstractmethod

class LLM(ABC):
    @abstractmethod
    def __init__(self, model_name: str = None, **kwargs) -> None:
        """Setup the model"""
        raise NotImplementedError

    @abstractmethod
    def __call__(
        self,
        prompt: str,
        max_tokens: int = 512,
        temperature: float = 0.0,
        top_logprobs: int = 5,  # NOTE: how many token logprobs per token position to return
    ) -> tuple[str, list[dict[str, float]]]:
        """Generate text from the model, and return the (text, logprobs) tuple
    
        Returns: (text: str, logprobs: list[dict[str, float]])
        Format of logprobs (suppose top_logprobs=5):
        [
            {<token_0>: <logprob>, <token_1>: <logprob>, ..., <token_4>: <logprob>},  # logprobs of top-5 tokens for position 0
            {<token_0>: <logprob>, <token_1>: <logprob>, ..., <token_4>: <logprob>},  # logprobs of top-5 tokens for position 1
            ...
        ]
        """
        raise NotImplementedError

# For example, we can implement OpenAI model endpoints as follows:
class OpenAILLM(LLM):
    def __init__(self, model_name: str = "gpt-4o-mini-2024-07-18", api_key: str = None) -> None:
        if not api_key:
            api_key = os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self.model_name = model_name

    def __call__(
        self,
        prompt: str,
        max_tokens: int = 512,
        temperature: float = 0.0,
        top_logprobs: int = 20,  # NOTE: how many token logprobs per token position to return
        **kwargs
    ) -> tuple[str, list[dict[str, float]]]:
        res = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            logprobs=True,
            top_logprobs=top_logprobs,
            **kwargs
        )
        text = res.choices[0].message.content
        logprobs = [  # NOTE: each token_position has <top_logprobs> token choices
            {choice.token: choice.logprob for choice in token_position.top_logprobs} for token_position in res.choices[0].logprobs.content
        ]
        return text, logprobs

# Usage example:
model_name = "gpt-4o-mini-2024-07-18"
api_key = Path("../apikeys/openai_bench.txt").read_text().strip()
llm = OpenAILLM(model_name, api_key)

text, logprobs = llm(prompt="Just say two tokens 'hello!'")

In [3]:
print(text)
print(f"There are {len(logprobs)} tokens in the text:")
print(f"Top-20 token logprobs of the first token position: {logprobs[0]}")
print(f"Top-20 token logprobs of the second token position: {logprobs[1]}")

hello!
There are 2 tokens in the text:
Top-20 token logprobs of the first token position: {'hello': -0.0019295862, 'Hello': -6.2519298, ' hello': -13.876929, 'Sure': -16.75193, 'hell': -18.00193, 'Certainly': -19.75193, 'hi': -19.87693, "I'm": -19.87693, '"': -20.00193, "'": -20.75193, '`': -21.00193, 'I': -21.12693, '你好': -21.75193, 'The': -22.12693, 'Here': -22.12693, '**': -22.25193, 'Sorry': -22.25193, 'sure': -22.37693, '``': -22.87693, ' Hello': -22.87693}
Top-20 token logprobs of the second token position: {'!': -1.2664457e-06, '!\n': -13.750001, '!\n\n': -15.625001, '!"': -19.000002, '!!': -19.375002, '！': -20.000002, ' !': -20.250002, '!</': -20.375002, "!'": -20.625002, '!【': -20.750002, '!*': -21.000002, '!)': -22.500002, '![': -22.500002, '!,': -22.750002, '!\\': -22.750002, '!\n\n\n': -22.750002, '%!': -22.875002, '!important': -22.875002, '<|end|>': -22.875002, '!!!': -23.000002}


## The Prompt for Asking User Support

In [19]:
prompt_template = """\
You are currently doing the question-answering (QA) task. Based on the information provided, you have to determine whether additional information is required for you to answer the question correctly.

Information provided (enclosed by triple backticks):
```
Question: {question}
Information: {information}
```

Answer a single word Yes if you need additional information to answer the question, and No otherwise.
Do you need additional information? Answer (Yes / No):"""

question = "Which company does Cheng-Kuang Wu work at?"

prompt_1 = prompt_template.format(
    question=question,
    information=""  # no information provided
)

prompt_2 = prompt_template.format(
    question=question,
    information="Cheng-Kuang Wu is a research scientist at Appier, Bppier, Cppier, Dppier, or Eppier."  # Vague information provided
)

prompt_3 = prompt_template.format(
    question=question,
    information="Cheng-Kuang Wu is a research scientist at Appier or Bppier."  # information provided
)

## Inference and Get "Need Hint" Probability

In [20]:
import numpy as np

def get_softmax_yes(logprob: dict[str, float]) -> float:
    """Get the softmaxed probability of the token 'Yes' among 'Yes' and 'No'"""
    TOKEN_LIST = ["Yes", "No"]
    probs = np.array([logprob[token] for token in TOKEN_LIST])
    softmax_probs = np.exp(probs) / np.sum(np.exp(probs))
    return softmax_probs[0]

res_1, logprobs_1 = llm(prompt=prompt_1)
res_2, logprobs_2 = llm(prompt=prompt_2)
res_3, logprobs_3 = llm(prompt=prompt_3)

In [27]:
print(f"Do you need hint for (prompt_1, prompt_2, prompt_3)?: {res_1, res_2, res_3}")
print(f"Need-user-support probability (softmax of 'Yes') for prompt_1: {get_softmax_yes(logprobs_1[0]) * 100:.2f}%")
print(f"Need-user-support probability (softmax of 'Yes') for prompt_2: {get_softmax_yes(logprobs_2[0]) * 100:.2f}%")
print(f"Need-user-support probability (softmax of 'Yes') for prompt_3: {get_softmax_yes(logprobs_3[0]) * 100:.2f}%")

Do you need hint for (prompt_1, prompt_2, prompt_3)?: ('Yes', 'Yes', 'No')
Need-user-support probability (softmax of 'Yes') for prompt_1: 100.00%
Need-user-support probability (softmax of 'Yes') for prompt_2: 98.90%
Need-user-support probability (softmax of 'Yes') for prompt_3: 22.27%
